<a href="https://colab.research.google.com/github/Raghuram-Veeramallu/Astro_Stat_Project2/blob/development/Group3_Project2_DataSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AST/STAT 5731 Project 2
### Research Synopsis
**Group 3**   
Daniel Warshofsky  
Hari Veeramallu  
Jacynda Alatoma  
Nicholas Kruegler

#### Research Question

*Synopsis here*


### Preamble

### Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# download the dataset to colab / local environment

## for google colab
!gdown 1LZts3qmmXzULOUMvNC97jIE9funj_BoK

## for local environment
# !gdown https://drive.google.com/uc?id=1LZts3qmmXzULOUMvNC97jIE9funj_BoK